In [19]:
#Import Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import gmaps

#Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [20]:
#Retrieve dataframe
path = "../Weather/cities.csv"
city_df = pd.read_csv(path)
city_df.head()

,City,Country,Date,Latitude,Longitude,High Temperature (F),Humidity,Cloudiness,Wind Speed
0,Faya,SA,1568912728,18.39,42.45,80.60,13,20,16.11
1,Juneau,US,1568912502,58.30,-134.42,53.60,93,90,8.05
2,Aswan,EG,1568912731,24.09,32.90,102.20,17,0,6.93
3,Albany,US,1568912440,42.65,-73.75,66.99,39,1,4.70
4,Ust-Omchug,RU,1568912734,61.13,149.63,32.45,96,99,4.83


In [18]:
#Create a heatmap of humidity of cities
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    city_df[["Latitude", "Longitude"]], weights=city_df['Humidity'],
    max_intensity=100, point_radius=3, dissipating=False
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
#Create a dataframe of cities with the ideal weather
ideal_df = city_df[(city_df['High Temperature (F)'] < 75) &
                   (city_df['High Temperature (F)'] > 60) &
                   (city_df['Wind Speed'] < 10) &
                   (city_df['Humidity'] < 40)].reset_index(drop=True)

In [17]:
#Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates

#Create a column for hotel name
ideal_df['Hotel Name'] = ""

ideal_df

,City,Country,Date,Latitude,Longitude,High Temperature (F),Humidity,Cloudiness,Wind Speed,Hotel Name
0,Albany,US,1568912440,42.65,-73.75,66.99,39,1,4.70,
1,Sfantu Gheorghe,RO,1568912760,44.65,26.83,68.00,37,0,6.93,
2,Kasempa,ZM,1568913028,-13.46,25.83,70.25,22,0,4.92,
3,Menongue,AO,1568913076,-14.66,17.68,71.69,14,16,4.68,
4,Buenos Aires,AR,1568912544,-34.61,-58.44,64.99,36,0,9.17,
5,Sabzevar,IR,1568913194,36.21,57.68,71.69,21,0,5.99,
6,Aliwal North,ZA,1568913203,-30.69,26.71,67.19,19,85,3.53,
7,Westport,US,1568913205,41.14,-73.36,69.01,34,1,3.36,
8,Pisco,PE,1568913313,-13.71,-76.20,71.60,30,90,8.05,
9,Chiredzi,ZW,1568913360,-21.05,31.67,73.85,36,0,5.21,


In [86]:
#Base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameters dictionary for api call
params = {
    'rankby': 'distance',
    'keyword': 'hotel',
    'key': gkey
}

#Create loop to retrieve latitude and longitude data and run api call
for index, row in ideal_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    
    #Add location to parameters
    params['location'] = f"{lat},{lon}"
    #Make request
    hotel_json = requests.get(base_url, params=params).json()
    
    #Add hotel name to dataframe
    if hotel_json['status'] == 'OK':
        ideal_df.loc[index, 'Hotel Name'] = hotel_json['results'][0]['name']
    else:
        pass

In [88]:
#Using template to add hotel markers to map
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
#Collect the hotel name, city and country from the dataframe
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]

In [94]:
#Create a map with markers for hotels
hotel_layer = gmaps.marker_layer(
    ideal_df[["Latitude", "Longitude"]],
    info_box_content=hotel_info
)

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [95]:
#Create a combined map
fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))